In [1]:
import pandas as pd

In [2]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Louisiana - Metadata/meta-Louisiana.json"
business_df = pd.read_json(path, lines = True)

In [ ]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Louisiana - Review/review-Louisiana.json"
review_df = pd.read_json(path, lines = True)

In [4]:
# Aggregate business metadata by gmap_id before merging
business_agg = business_df.groupby("gmap_id", as_index=False).agg({
    "name": "first",  # Keep first occurrence (assumes same name per gmap_id)
    "address": "first",
    "description": "first",
    "latitude": "first",
    "longitude": "first",
    "category": lambda x: list(set(x.sum())) if x.apply(lambda v: isinstance(v, list)).any() else list(set(x)),
    "avg_rating": "mean",  # Use mean rating if multiple exist
    "num_of_reviews": "sum",
    "price": "first",
    "hours": "first",
    "MISC": "first",
    "state": "first",
    "relative_results": "first",
    "url": "first"
})

In [5]:
# Merge with aggregated business data
merged_df = review_df.merge(business_agg, on="gmap_id", how="left")

In [6]:
#remove rows/records where 'text' is null
merged_df = merged_df.dropna(subset = ['text'])
#check if null values in 'text' column still exist
print(merged_df['text'].isna().sum())

0


In [7]:
#delete pics 
merged_df = merged_df.drop(columns = ['pics'])

In [8]:
negative_df = merged_df[merged_df['rating'].isin([1,2])]
negative_df

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1,1.049434e+20,Avaviel —,1611943181664,1.0,"""We're sorry we don't have a locksmith serving...","{'time': 1611945977279, 'text': 'Hello Avaviel...",0x86312d61f5ac28b5:0x5e6c55690b65096a,KeyMe Locksmiths,"KeyMe Locksmiths, 6652 Youree Dr, Shreveport, ...",None,...,-93.725300,"[Key duplication service, Security system inst...",4.0,34,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",None,Open 24 hours,"[0x8636d3d924fc5dfb:0x24ad8c61ee4e54ce, 0x8636...",https://www.google.com/maps/place//data=!4m2!3...
7,1.107463e+20,Ben Mahoney,1579581685975,1.0,Was looking for a locksmith. Found a vending m...,"{'time': 1582924403560, 'text': 'Ben , thanks...",0x86312d61f5ac28b5:0x5e6c55690b65096a,KeyMe Locksmiths,"KeyMe Locksmiths, 6652 Youree Dr, Shreveport, ...",None,...,-93.725300,"[Key duplication service, Security system inst...",4.0,34,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",None,Open 24 hours,"[0x8636d3d924fc5dfb:0x24ad8c61ee4e54ce, 0x8636...",https://www.google.com/maps/place//data=!4m2!3...
202,1.038514e+20,Ari Wilson,1583363957125,2.0,Called regarding an order one of the staff hel...,None,0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,-93.762363,"[Medical supply store, Wheelchair rental service]",4.3,50,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...",{'From the business': ['Identifies as women-le...,Open ⋅ Closes 5PM,"[0x8636d2e70c512615:0x891bd4b4ec40cfb9, 0x8631...",https://www.google.com/maps/place//data=!4m2!3...
204,1.100255e+20,Flournoy Group,1579662881780,1.0,This company is taking advantage of the custom...,"{'time': 1579704185731, 'text': 'Thank you for...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,-93.762363,"[Medical supply store, Wheelchair rental service]",4.3,50,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...",{'From the business': ['Identifies as women-le...,Open ⋅ Closes 5PM,"[0x8636d2e70c512615:0x891bd4b4ec40cfb9, 0x8631...",https://www.google.com/maps/place//data=!4m2!3...
206,1.035609e+20,Rachel,1518478716949,2.0,Staff could have been more professional and a ...,"{'time': 1520434288735, 'text': 'Rachel, I am ...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,-93.762363,"[Medical supply store, Wheelchair rental service]",4.3,50,None,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu...",{'From the business': ['Identifies as women-le...,Open ⋅ Closes 5PM,"[0x8636d2e70c512615:0x891bd4b4ec40cfb9, 0x8631...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7535719,1.076609e+20,Tracie Burke,1597924172823,1.0,Rip off.,None,0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,-90.080259,[Tourist attraction],4.9,938,None,"[[Wednesday, 7AM–10PM], [Thursday, 7AM–10PM], ...",{'Amenities': ['Good for kids']},None,"[0x862099caaea6dddd:0x538e347f2f01b65e, 0x8620...",https://www.google.com/maps/place//data=!4m2!3...
7535732,1.162832e+20,Alexander Horn,1535208504894,2.0,Used to be alot better. Guide did not go much ...,"{'time': 1535738640824, 'text': 'WE ARE VERY S...",0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,-90.080259,[Tourist attraction],4.9,938,None,"[[Wednesday, 7AM–10PM], [Thursday, 7AM–10PM], ...",{'Amenities': ['Good for kids']},None,"[0x862099caaea6dddd:0x538e347f2f01b65e, 0x8620...",https://www.google.com/maps/place//data=!4m2!3...
7535739,1.112577e+20,Ken Ovryn,1557946604101,2.0,Sexist boat operator is a Trump supporter prea...,None,0x862098b14602cd67:0xc1f

In [9]:
import ast

negative_df["category"] = negative_df["category"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expand the list into separate columns
category_expanded = negative_df["category"].apply(pd.Series)

# Rename columns as 'category_1', 'category_2', etc.
category_expanded.columns = [f"category_{i+1}" for i in range(category_expanded.shape[1])]

# Concatenate with original dataset
negative_df = pd.concat([negative_df, category_expanded], axis=1)

# Drop original 'category' column
#cleaned_file.drop(columns=["category"], inplace=True)

negative_df

C:\Users\eliza\AppData\Local\Temp\ipykernel_30940\761791609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df["category"] = negative_df["category"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13
1,1.049434e+20,Avaviel —,1611943181664,1.0,"""We're sorry we don't have a locksmith serving...","{'time': 1611945977279, 'text': 'Hello Avaviel...",0x86312d61f5ac28b5:0x5e6c55690b65096a,KeyMe Locksmiths,"KeyMe Locksmiths, 6652 Youree Dr, Shreveport, ...",None,...,Emergency locksmith service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.107463e+20,Ben Mahoney,1579581685975,1.0,Was looking for a locksmith. Found a vending m...,"{'time': 1582924403560, 'text': 'Ben , thanks...",0x86312d61f5ac28b5:0x5e6c55690b65096a,KeyMe Locksmiths,"KeyMe Locksmiths, 6652 Youree Dr, Shreveport, ...",None,...,Emergency locksmith service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,1.038514e+20,Ari Wilson,1583363957125,2.0,Called regarding an order one of the staff hel...,None,0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,1.100255e+20,Flournoy Group,1579662881780,1.0,This company is taking advantage of the custom...,"{'time': 1579704185731, 'text': 'Thank you for...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,1.035609e+20,Rachel,1518478716949,2.0,Staff could have been more professional and a ...,"{'time': 1520434288735, 'text': 'Rachel, I am ...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7535719,1.076609e+20,Tracie Burke,1597924172823,1.0,Rip off.,None,0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7535732,1.162832e+20,Alexander Horn,1535208504894,2.0,Used to be alot better. Guide did not go much ...,"{'time': 1535738640824, 'text': 'WE ARE VERY S...",0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7535739,1.112577e+20,Ken Ovryn,1557946604101,2.0,Sexist boat operator is a Trump supporter prea...,None,0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7535768,1.033892e+20,JM Davis,1477752554485,2.0,Although we enjoyed our airboat ride with New ...,None,0x862098b14602cd67:0xc1f8cf38232cd993,"New Orleans Airboat Tours, LLC","New Orleans Airboat Tours, LLC, 4757 Orleans W...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import re

In [11]:
healthcare_categories = {
    "medical clinic", "women's health clinic", "home health care service", "internist", "allergist",
    "diabetes center", "doctor", "family practice physician", "optometrist", "pharmacy", "dentist",
    "medical supply store", "orthopedic surgeon", "surgeon", "psychotherapist", "dental clinic",
    "orthodontist", "physical therapist", "chiropractor", "hospital", "sleep clinic", "optician",
    "cancer treatment center", "medical center", "physical therapy clinic", "medical laboratory",
    "public health department", "psychologist", "radiologist", "podiatrist", "nursing home",
    "medical group", "x-ray lab", "dermatologist", "pediatrician", "massage therapist",
    "periodontist", "obstetrician-gynecologist", "sports medicine clinic", "psychiatric hospital",
    "orthotics & prosthetics service", "physical fitness program", "ophthalmologist", "hospice",
    "retirement home", "rehabilitation center", "pediatric dentist", "teeth whitening service",
    "dental hygienist", "endodontist", "dental implants periodontist", "cosmetic dentist",
    "emergency dental service", "urgent care center", "wellness center", "walk-in clinic",
    "general practitioner", "children's hospital", "mental health clinic", "otolaryngologist"
}
# Function to check if a business belongs to the healthcare category
def classify_healthcare(row):
    for i in range(1, 16):  # Assuming category_1 to category_15 exist
        category_col = f"category_{i}"
        if category_col in row and isinstance(row[category_col], str):  # Check if column exists and is a string
            for keyword in healthcare_categories:
                if re.search(rf"\b{keyword}\b", row[category_col], re.IGNORECASE):  # Match whole word
                    return "Healthcare"  # Assign healthcare label
    return "Other"  # Default category if no match

# Apply the function to create a new 'general_labelling' column
negative_df["general_labelling"] = negative_df.apply(classify_healthcare, axis=1)

In [12]:
# Filter records where 'general_labelling' is "Healthcare"
healthcare_dataset = negative_df[negative_df["general_labelling"] == "Healthcare"]

# Define the file path to save the Healthcare dataset
louisiana_healthcare_dataset_path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/louisiana_healthcare_dataset.csv"

# Save the filtered dataset to CSV
healthcare_dataset.to_csv(louisiana_healthcare_dataset_path, index=False)

In [14]:
louisiana_healthcare_dataset_path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/louisiana_healthcare_dataset.csv"
df = pd.read_csv(louisiana_healthcare_dataset_path)
df

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,category_13,general_labelling
0,1.038514e+20,Ari Wilson,1583363957125,2.0,Called regarding an order one of the staff hel...,NaN,0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
1,1.100255e+20,Flournoy Group,1579662881780,1.0,This company is taking advantage of the custom...,"{'time': 1579704185731, 'text': 'Thank you for...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
2,1.035609e+20,Rachel,1518478716949,2.0,Staff could have been more professional and a ...,"{'time': 1520434288735, 'text': 'Rachel, I am ...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
3,1.026735e+20,Marcinho Savant,1461953425690,1.0,"In my experience this morning, I found them no...",NaN,0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
4,1.068112e+20,Allison Wall,1534783072584,1.0,"This place is a joke, they refused to sell us ...","{'time': 1534867614097, 'text': 'Allison, I am...",0x8636d33deda334eb:0x8539697e533cad66,Accucare Medical,"Accucare Medical, 9011 Linwood Ave, Shreveport...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16273,1.009026e+20,Mark S,1424721229804,1.0,Brought my dog in on a Friday because he was v...,NaN,0x8626ba564f365f59:0x7e6e5bdbec491a49,Azalea Lakes Veterinary Clinic,"Azalea Lakes Veterinary Clinic, 15225 Jefferso...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
16274,1.037343e+20,Kaycie LaGrone,1401746019817,1.0,I boarded my dog there while I was on a trip a...,NaN,0x8626ba564f365f59:0x7e6e5bdbec491a49,Azalea Lakes Veterinary Clinic,"Azalea Lakes Veterinary Clinic, 15225 Jefferso...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
16275,1.016109e+20,Edie Chaffin,1272678848478,1.0,I brought my lab there for years. The past yea...,NaN,0x8626ba564f365f59:0x7e6e5bdbec491a49,Azalea Lakes Veterinary Clinic,"Azalea Lakes Veterinary Clinic, 15225 Jefferso...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
16276,1.174574e+20,Jon Goodart,1560367162642,1.0,I would say the coaches and workouts are great...,"{'time': 1573252655408, 'text': ""Sorry for the...",0x8626a38aeb6e628f:0xce96adca225b4502,Orangetheory Fitness,"Orangetheory Fitness, 2373 Town Center Blvd #7...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
